<a href="https://colab.research.google.com/github/NurFaizin/ann-backpropagation-regression/blob/main/annts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [2]:
def relu(z):
    a = np.maximum(0,z)
    return a

In [24]:
def initialize_params(layer_sizes):
    params = {}
    for i in range(1, len(layer_sizes)):
        params['W' + str(i)] = np.random.randn(layer_sizes[i], layer_sizes[i-1])*0.01
        params['B' + str(i)] = np.random.randn(layer_sizes[i],1)*0.01
    print(params)
    return params

In [4]:
def forward_propagation(X_train, params):
    layers = len(params)//2
    values = {}
    for i in range(1, layers+1):
        if i==1:
            values['Z' + str(i)] = np.dot(params['W' + str(i)], X_train) + params['B' + str(i)]
            values['A' + str(i)] = relu(values['Z' + str(i)])
        else:
            values['Z' + str(i)] = np.dot(params['W' + str(i)], values['A' + str(i-1)]) + params['B' + str(i)]
            if i==layers:
                values['A' + str(i)] = values['Z' + str(i)]
            else:
                values['A' + str(i)] = relu(values['Z' + str(i)])
    return values

In [5]:
def compute_cost(values, Y_train):
    layers = len(values)//2
    Y_pred = values['A' + str(layers)]
    cost = 1/(2*len(Y_train)) * np.sum(np.square(Y_pred - Y_train))
    return cost

In [6]:
def backward_propagation(params, values, X_train, Y_train):
    layers = len(params)//2
    m = len(Y_train)
    grads = {}
    for i in range(layers,0,-1):
        if i==layers:
            dA = 1/m * (values['A' + str(i)] - Y_train)
            dZ = dA
        else:
            dA = np.dot(params['W' + str(i+1)].T, dZ)
            dZ = np.multiply(dA, np.where(values['A' + str(i)]>=0, 1, 0))
        if i==1:
            grads['W' + str(i)] = 1/m * np.dot(dZ, X_train.T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
        else:
            grads['W' + str(i)] = 1/m * np.dot(dZ,values['A' + str(i-1)].T)
            grads['B' + str(i)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    return grads


In [7]:
def update_params(params, grads, learning_rate):
    layers = len(params)//2
    params_updated = {}
    for i in range(1,layers+1):
        params_updated['W' + str(i)] = params['W' + str(i)] - learning_rate * grads['W' + str(i)]
        params_updated['B' + str(i)] = params['B' + str(i)] - learning_rate * grads['B' + str(i)]
    return params_updated

In [8]:
def model(X_train, Y_train, layer_sizes, num_iters, learning_rate):
    params = initialize_params(layer_sizes)
    for i in range(num_iters):
        values = forward_propagation(X_train.T, params)
        cost = compute_cost(values, Y_train.T)
        grads = backward_propagation(params, values,X_train.T, Y_train.T)
        params = update_params(params, grads, learning_rate)
        print('Cost at iteration ' + str(i+1) + ' = ' + str(cost) + '\n')
    return params

In [9]:
def compute_accuracy(X_train, X_test, Y_train, Y_test, params):
    values_train = forward_propagation(X_train.T, params)
    values_test = forward_propagation(X_test.T, params)
    train_acc = mean_squared_error(Y_train, values_train['A' + str(len(layer_sizes)-1)].T)
    test_acc = mean_squared_error(Y_test, values_test['A' + str(len(layer_sizes)-1)].T)
    return train_acc, test_acc

In [10]:
def predict(X, params):
    values = forward_propagation(X.T, params)
    predictions = values['A' + str(len(values)//2)].T
    return predictions

In [25]:
data = load_boston()                                                              #load dataset
X,Y = data["data"], data["target"]                                                #separate data into input and output features
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size = 0.2)           #split data into train and test sets in 80-20 ratio

In [22]:
layer_sizes = [13, 5, 5, 1]                                                       #set layer sizes, do not change the size of the first and last layer 
num_iters = 10                                                                  #set number of iterations over the training set(also known as epochs in batch gradient descent context)
learning_rate = 0.05                                                              #set learning rate for gradient descent

In [26]:
params = model(X_train, Y_train, layer_sizes, num_iters, learning_rate)           #train the model
train_acc, test_acc = compute_accuracy(X_train, X_test, Y_train, Y_test, params)  #get training and test accuracy

{'W1': array([[-0.01987646, -0.01648039, -0.00812933, -0.00314752, -0.0014914 ,
        -0.00018889, -0.00068595,  0.00027011,  0.01337197,  0.00985507,
        -0.01054339, -0.0106918 ,  0.00382479],
       [ 0.00700548, -0.02824674,  0.01158062, -0.00304319,  0.00338397,
         0.00694075,  0.00172341, -0.0041179 ,  0.008439  ,  0.00627464,
        -0.00475992,  0.00792243,  0.00394957],
       [ 0.0030709 , -0.01831788,  0.00779384, -0.00204208, -0.00356374,
         0.01918188,  0.00075539,  0.00820069, -0.00444025, -0.0033358 ,
         0.00591393, -0.00123545, -0.01022899],
       [ 0.0007677 ,  0.00219899, -0.011167  , -0.00411237, -0.00895719,
        -0.00110955, -0.00835805, -0.00874897,  0.00228144, -0.01106464,
         0.00051064, -0.00375546, -0.02357263],
       [-0.01358749,  0.00762688, -0.01360596, -0.01181987, -0.00725035,
        -0.0038445 , -0.00129477,  0.00378545,  0.00707509, -0.00370555,
         0.00253246, -0.00338061, -0.00152293]]), 'B1': array([[-0.0022

In [21]:
print('Root Mean Squared Error on Training Data = ' + str(train_acc))
print('Root Mean Squared Error on Test Data = ' + str(test_acc))

Root Mean Squared Error on Training Data = 76.9588299913496
Root Mean Squared Error on Test Data = 72.41929366835697
